<img align="centre" src="../../Supplementary_data/Github_banner.jpg" width="100%">

# Enhanced Combined Drought Index - Temperature Drought Index

* **Products used:** 
[ls5_st](https://explorer.digitalearth.africa/products/ls5_st),
[ls7_st](https://explorer.digitalearth.africa/products/ls7_st),
[ls8_st](https://explorer.digitalearth.africa/products/ls8_st),
[ls9_st](https://explorer.digitalearth.africa/products/ls9_st)

## Background 

Drought is an extended period, during which, fresh water availability and accessibility for the ecosystem at a given time and place is below normal, due to unfavourable spatial and temporal distribution of rainfall, temperature, soil moisture and wind characteristics [(Balint et al., 2013)](https://doi.org/10.1016/B978-0-444-59559-1.00023-2). Severe droughts can affect large populations, leading to a long-term threat to people’s livelihoods and result in tremendous economic loss [(Enenkel et al., 2016)](https://doi.org/10.3390/rs8040340).

The **Enhanced Drought Index** is a combination of the following components: Vegetation, Precipitation, Temperature and Soil moisture.

This notebook will focus on the **Temperature** component, which considers temperature excesses and persistence of high temperatures. The index based on temperature is named as **Temperature Drought Index (TDI)** The Temperature Drought Index (TDI) is used to assess the combined impact of temperature and precipitation deficits on drought conditions. It is particularly useful for understanding the role of temperature in exacerbating drought effects, as high temperatures can increase evapotranspiration (the combined loss of water through evaporation and plant transpiration), which in turn can worsen drought conditions.

The notebook outlines:

***
1. Loading Python and DE Africa Packages
2. Setting dask cluster
3. Loading area on interest
5. Load Landsat Surface temperature
6. Resampling of data to dekadal (10-day) intervals
7. Calculate the Vegetation Drought Index
8. Export the result as zip

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages

In [1]:
import json
import os
from datetime import datetime, timedelta

import dask
import geopandas as gpd
import numpy as np
import pandas as pd
import rioxarray
import toolz
import xarray as xr
import datacube
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.dask import create_local_dask_cluster
from deafrica_tools.datahandling import load_ard
from dekad import group_by_dekad, bin_by_interest_period, get_dekad_no_in_year, get_no_data_mask, max_consecutive_ones
from odc.geo.geobox import GeoBox
from odc.geo.geom import Geometry
from odc.geo.xr import rasterize
from pyarrow import Table
from pyarrow.parquet import write_table

### Connect to the datacube

Connect to the datacube so we can access DE Africa data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [2]:
# Connect to the datacube
dc = datacube.Datacube(app="TemperatureDroughtIndex")

### Set up a Dask cluster

Dask can be used to better manage memory use and conduct the analysis in parallel. 
For an introduction to using Dask with Digital Earth Africa, see the [Dask notebook](../../Beginners_guide/08_Parallel_processing_with_dask.ipynb).

>**Note**: We recommend opening the Dask processing window to view the different computations that are being executed; to do this, see the *Dask dashboard in DE Africa* section of the [Dask notebook](../../Beginners_guide/08_Parallel_processing_with_dask.ipynb).

To use Dask, set up the local computing cluster using the cell below.

In [3]:
create_local_dask_cluster()

/opt/venv/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44891 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/mickwelli@bigpond.com/proxy/44891/status,
Dashboard: /user/mickwelli@bigpond.com/proxy/44891/status,Workers: 1
Total threads: 2,Total memory: 11.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37805,Workers: 1
Dashboard: /user/mickwelli@bigpond.com/proxy/44891/status,Total threads: 2
Started: Just now,Total memory: 11.21 GiB
Comm: tcp://127.0.0.1:44117,Total threads: 2
Dashboard: /user/mickwelli@bigpond.com/proxy/36361/status,Memory: 11.21 GiB
Nanny: tcp://127.0.0.1:42599,


### Analysis parameters

The following cell sets important parameters for the analysis:

* `resolution`: The x and y cell resolution of the satellite data in metres (spatial resolution). We'll use 5,000 m, which is approximately equal to the default CHIRPS resolution.
* `output_crs` : The coordinate reference system that the loaded data is to be reprojected to.
* `dask_chunks`:  the size of the dask chunks, dask breaks data into manageable chunks that can be easily stored in memory, e.g. `dict(x=1000,y=1000)`
* `time_range` : Time range to load data for.
* `ip` : The interest period to use to calculate the drought indices e.g. (3, 4, 5 dekads). It defines to what extent past observations are considered. Longer IPs detect more severe drought events. For example, if the IP=3 dekads, then the drought index (say PDI) of 0.35 for dekad 2 of 2006 implies actual drought for dekad 36 of 2005, dekad 1 of 2006 and dekad 2 of 2006.
* `output_dir` :  The directory in which to store results from the analysis.

**If running the notebook for the first time**, keep the default settings below.
This will demonstrate how the analysis works and provide meaningful results.

In [4]:
resolution = (-5000, 5000)
output_crs = "EPSG:6933"
dask_chunks = dict(time=366)
time_range = ("2014","2025")
output_dir = "results"
# Corresponding to the six-months Standardized Precipitation Evapotranspiration Index
ip = 18

# Create the outpur directory if it does not exist.
os.makedirs(output_dir, exist_ok=True)

Below, a vector dataset is called and read which contains the extents of level 1 administrative boundaries in Ethiopia. In the default case, the regional state of Amhara is northern Ethiopia is the area of interest. This can be replaced with any administrative boundary, such as those available on the [GADM database](https://gadm.org/data.html). Larger areas have a larger processing and memory burden and may require additional resources (larger instances) to run effectively.

In [5]:
# Read GADM Ethiopia level 1 file
eth = gpd.read_file('data/gadm41_ETH_1.json')

# Select the country of interest by providing the name below
idx = eth[eth['NAME_1'] == 'Amhara'].index[0]
geopolygon = Geometry(geom=eth.iloc[idx].geometry, crs=eth.crs)

# Create the output path
output_path = os.path.join(output_dir, f"TDI_Amhara.parquet")

## Load Landsat surface temperature data
Load the Landsat surface temperature data from the datacube using the analysis parameters set in the previous section.

In [6]:
# Load the Landsat data
ds_ls_st = load_ard(dc=dc,
                    products=["ls8_st", "ls9_st"],
                    measurements=["surface_temperature"],
                    min_gooddata=0.9,
                    time=time_range,
                    dask_chunks=dict(time=366),
                    geopolygon=geopolygon,
                    resolution=resolution,
                    output_crs=output_crs,
                   )
                    
ds_ls_st

/opt/venv/lib/python3.12/site-packages/deafrica_tools/datahandling.py:244: UserWarning: Setting 'min_gooddata' percentage to > 0.0 will cause dask arrays to compute when loading pixel-quality data to calculate 'good pixel' percentage. This can slow the return of your dataset.
  warnings.warn(


Using pixel quality parameters for USGS Collection 2
Finding datasets
    ls8_st
    ls9_st
Counting good quality pixels for each time step


/opt/venv/lib/python3.12/site-packages/distributed/client.py:3361: UserWarning: Sending large graph of size 16.34 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Filtering to 4449 out of 4538 time steps with at least 90.0% good quality pixels
Applying pixel quality/cloud mask
Re-scaling Landsat C2 data
Returning 4449 time steps as a dask array


<xarray.Dataset> Size: 219MB
Dimensions:              (time: 4449, y: 128, x: 96)
Coordinates:
  * time                 (time) datetime64[ns] 36kB 2014-01-02T07:40:34.91350...
  * y                    (y) float64 1kB 1.742e+06 1.738e+06 ... 1.108e+06
  * x                    (x) float64 768B 3.402e+06 3.408e+06 ... 3.878e+06
    spatial_ref          int32 4B 6933
Data variables:
    surface_temperature  (time, y, x) float32 219MB dask.array<chunksize=(349, 128, 96), meta=np.ndarray>
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref

### Masked the landsat data

In [7]:
# Rasterize the country geopolygon and mask the loaded Landsat data.
country_mask = rasterize(poly=geopolygon, how=ds_ls_st.odc.geobox)
ds_ls_st = ds_ls_st.where(country_mask)

In [8]:
# Convert from Kelvins to Celsius
da_ls_st = ds_ls_st["surface_temperature"] - 273.15
da_ls_st

<xarray.DataArray 'surface_temperature' (time: 4449, y: 128, x: 96)> Size: 219MB
dask.array<sub, shape=(4449, 128, 96), dtype=float32, chunksize=(349, 128, 96), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 36kB 2014-01-02T07:40:34.913504 ... 20...
  * y            (y) float64 1kB 1.742e+06 1.738e+06 ... 1.112e+06 1.108e+06
  * x            (x) float64 768B 3.402e+06 3.408e+06 ... 3.872e+06 3.878e+06
    spatial_ref  int32 4B 6933

### Resample data

Resample the surface temperature `da_ls_st` timeseries into **dekadal** (10-day) timesteps.

In [9]:
%%time
# Resample the surface temperature data from daily to decadal (10-day) intervals
da_ls_st_dekadal = group_by_dekad(da_ls_st).mean(dim="time").compute()
da_ls_st_dekadal

/opt/venv/lib/python3.12/site-packages/distributed/client.py:3361: UserWarning: Sending large graph of size 17.07 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


CPU times: user 52.5 s, sys: 3.1 s, total: 55.6 s
Wall time: 12min 56s


<xarray.DataArray 'surface_temperature' (dekad: 409, y: 128, x: 96)> Size: 20MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * y            (y) float64 1kB 1.742e+06 1.738e+06 ... 1.112e+06 1.108e+06
  * x            (x) float64 768B 3.402e+06 3.408e+06 ... 3.872e+06 3.878e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 3kB 2014-01-01 2014-01-11 ... 2025-05-11

### Modify the timeseries

The raw timeseries of surface temperature is modified to adjust the range of all variables to avoid a division by zero.

In [10]:
da_ls_st_dekadal_modified = (da_ls_st_dekadal.max() + 1) - da_ls_st_dekadal

### Group the modified timeseries using the `ip` parameter

The `ip` parameter determines to what extent past observations are considered. Longer IPs detect more severe drought events. The default `ip` used of **18 dekads** corresponds to the 6-month Standardized Precipitation Evapotranspiration Index.

For example, calculating the Vegetation Drought Index for the dekad `2011-01-11` using an interest period of `ip=3` requires rainfall data from the dekad `2010-12-21`, `2011-01-01` and `2011-01-11`.


**Each interest period is labelled using its end dekad.**

In [11]:
# Group the modified surface temperature dekadal timeseries by the interest period.
# The dictionary keys are the end dekad for each interest period.
# The values of the dictionary are the surface temperature data that belongs to the interest period
da_ls_st_binned_by_interest_period = bin_by_interest_period(ds=da_ls_st_dekadal_modified, ip=ip)

In [12]:
# Get the interest periods
interest_periods = list(da_ls_st_binned_by_interest_period.keys())

### Get the average values for each interest period

Get the actual average surface temperature for each interest period.

In [13]:
da_ls_st_actual_avg_for_ip = xr.concat([da_ls_st_binned_by_interest_period[i].mean(dim="dekad").assign_coords(dekad=i).expand_dims({"dekad": 1}) for i in interest_periods], dim="dekad")
da_ls_st_actual_avg_for_ip

<xarray.DataArray 'surface_temperature' (dekad: 410, y: 128, x: 96)> Size: 40MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * y            (y) float64 1kB 1.742e+06 1.738e+06 ... 1.112e+06 1.108e+06
  * x            (x) float64 768B 3.402e+06 3.408e+06 ... 3.872e+06 3.878e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 3kB 2014-01-01 2014-01-11 ... 2025-05-11

### Get the long term average for each interest period over the years of available data


In [14]:
# Group the interest periods by year by getting the dekad number in the year for the end dekad of
# the interest period. 
grouped_by_year = toolz.groupby(lambda dekad: get_dekad_no_in_year(date=dekad), interest_periods)

Get the long term average surface temperature for each interest period.

In [15]:
long_term_avg_ls_st = {}
for dekad_no_in_year, interest_periods_list in grouped_by_year.items():
    long_term_avg_for_period = xr.concat([da_ls_st_binned_by_interest_period[i] for i in interest_periods_list], dim="dekad").mean(dim="dekad")
    long_term_avg_ls_st[dekad_no_in_year] = long_term_avg_for_period
    
da_ls_st_long_term_avg_for_ip = xr.concat([long_term_avg_ls_st[get_dekad_no_in_year(i)].assign_coords(dekad=i).expand_dims({"dekad": 1}) for  i in interest_periods], dim="dekad")

assert all(da_ls_st_actual_avg_for_ip.dekad.values == da_ls_st_long_term_avg_for_ip.dekad.values)

da_ls_st_long_term_avg_for_ip

<xarray.DataArray 'surface_temperature' (dekad: 410, y: 128, x: 96)> Size: 40MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * y            (y) float64 1kB 1.742e+06 1.738e+06 ... 1.112e+06 1.108e+06
  * x            (x) float64 768B 3.402e+06 3.408e+06 ... 3.872e+06 3.878e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 3kB 2014-01-01 2014-01-11 ... 2025-05-11

### Get the actual length of continous excess in each interest period

For surface temperature the run length is the number of successive dekads in an interest period above the long term average for the same interest period.

In [16]:
%%time
# For each interest period get the maximum number of successive dekads above long term average surface temperature in the interest period.
ls_st_run_length = []
for interest_period in interest_periods:
    # Get the dekads in the interest period
    ds = da_ls_st_binned_by_interest_period[interest_period]
    
    # Identify pixels which are empty for all dekads
    no_data_mask = xr.apply_ufunc(
        get_no_data_mask,
        ds,
        input_core_dims=[["dekad"]],
        vectorize=True,
        dask="allowed",
    )
    
    # Get the long term average for the interest period.
    long_term_avg = da_ls_st_long_term_avg_for_ip.sel(dekad=interest_period)
    
    # Get the pixels where the surface temperature is above the long term average surface temperature
    mask = xr.where(ds > long_term_avg, 1, 0)
    
    # Get the maximum number of successive dekads above long term average surface temperature.
    actual_run_length = xr.apply_ufunc(
        max_consecutive_ones,
        mask,
        input_core_dims=[["dekad"]],
        vectorize=True,
        dask="allowed",
    )
    
    # Modify the run length
    modified_run_length = (actual_run_length.max() + 1) - actual_run_length
    modified_run_length = modified_run_length.where(~no_data_mask)
    ls_st_run_length.append(modified_run_length.assign_coords(dekad=interest_period).expand_dims({"dekad": 1}))

da_ls_st_actual_run_length = xr.concat(ls_st_run_length, dim="dekad")
da_ls_st_actual_run_length

CPU times: user 1min 2s, sys: 899 ms, total: 1min 3s
Wall time: 1min 4s


<xarray.DataArray 'surface_temperature' (dekad: 410, y: 128, x: 96)> Size: 40MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
    spatial_ref  int32 4B 6933
  * y            (y) float64 1kB 1.742e+06 1.738e+06 ... 1.112e+06 1.108e+06
  * x            (x) float64 768B 3.402e+06 3.408e+06 ... 3.872e+06 3.878e+06
  * dekad        (dekad) datetime64[ns] 3kB 2014-01-01 2014-01-11 ... 2025-05-11

### Get the long term average of continous excess in each interest period

In [17]:
# Get the long term average surface temperature run length for each interest period 
long_term_avg_run_lenth_ls_st = {}
for dekad_no_in_year, interest_periods_list in grouped_by_year.items():
    long_term_avg_for_period = da_ls_st_actual_run_length.sel(dekad=interest_periods_list).mean(dim="dekad")
    long_term_avg_run_lenth_ls_st[dekad_no_in_year] = long_term_avg_for_period
    
da_ls_st_long_term_avg_run_length_for_ip = xr.concat([long_term_avg_run_lenth_ls_st[get_dekad_no_in_year(i)].assign_coords(dekad=i).expand_dims({"dekad": 1}) for  i in interest_periods], dim="dekad")
da_ls_st_long_term_avg_run_length_for_ip

<xarray.DataArray 'surface_temperature' (dekad: 410, y: 128, x: 96)> Size: 40MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
    spatial_ref  int32 4B 6933
  * y            (y) float64 1kB 1.742e+06 1.738e+06 ... 1.112e+06 1.108e+06
  * x            (x) float64 768B 3.402e+06 3.408e+06 ... 3.872e+06 3.878e+06
  * dekad        (dekad) datetime64[ns] 3kB 2014-01-01 2014-01-11 ... 2025-05-11

### Calculate the Temperature Drought Index

In [18]:
TDI = (da_ls_st_actual_avg_for_ip / da_ls_st_long_term_avg_for_ip) * np.sqrt((da_ls_st_actual_run_length / da_ls_st_long_term_avg_run_length_for_ip))
TDI

<xarray.DataArray 'surface_temperature' (dekad: 410, y: 128, x: 96)> Size: 40MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * y            (y) float64 1kB 1.742e+06 1.738e+06 ... 1.112e+06 1.108e+06
  * x            (x) float64 768B 3.402e+06 3.408e+06 ... 3.872e+06 3.878e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 3kB 2014-01-01 2014-01-11 ... 2025-05-11

In [19]:
#scaled TDI
TDI_scaled = (TDI - TDI.min()) / (TDI.max() - TDI.min())
TDI_scaled

<xarray.DataArray 'surface_temperature' (dekad: 410, y: 128, x: 96)> Size: 40MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * y            (y) float64 1kB 1.742e+06 1.738e+06 ... 1.112e+06 1.108e+06
  * x            (x) float64 768B 3.402e+06 3.408e+06 ... 3.872e+06 3.878e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 3kB 2014-01-01 2014-01-11 ... 2025-05-11

In [20]:
# Convert to DataFrame.
df = TDI_scaled.to_dataframe().drop(columns="spatial_ref")
df

surface_temperature
dekad      y         x                             
2014-01-01 1742500.0 3402500.0                  NaN
                     3407500.0                  NaN
                     3412500.0                  NaN
                     3417500.0                  NaN
                     3422500.0                  NaN
...                                             ...
2025-05-11 1107500.0 3857500.0                  NaN
                     3862500.0                  NaN
                     3867500.0                  NaN
                     3872500.0                  NaN
                     3877500.0                  NaN

[5038080 rows x 1 columns]

In [21]:
# Convert the DataFrame to an Arrow table
table = Table.from_pandas(df)

# Get the tables existing metadata
existing_meta = table.schema.metadata

# Dump the crs of the DataArray as new metadata to JSON.
meta_json = json.dumps(dict(crs=str(TDI_scaled.rio.crs)))

# Combine the metadata
combined_meta = {
b"xarray.metadata": meta_json.encode(),
**existing_meta,
}

# Replace the metadata.
table = table.replace_schema_metadata(combined_meta)

### Export the results in a Zip file

In [22]:
# Write the table
write_table(table, output_path, compression="GZIP")

---

## Additional information

<b> License </b> The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0).

Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

<b> Contact </b> If you need assistance, please post a question on the [DE Africa Slack channel](https://digitalearthafrica.slack.com/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).

If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

<b> Compatible datacube version </b>

In [23]:
print(datacube.__version__)

1.8.20


**Last Tested:**

In [24]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2025-05-19'